# Example pygccxml notebook on Binder

Running this notebook on Binder allows you to execute the code online.

Please note that provisioning may take about 1-2 minutes.

<a target="_doc" href="https://mybinder.org/v2/gh/EricCousineau-TRI/pygccxml/feature-py-notebook-example?filepath=pygccxml/docs/examples/notebook/example.ipynb">
  <img src="https://mybinder.org/badge_logo.svg"/>
</a>

The following example shows an example usage of `pygccxml` on Ubuntu
Bionic, from within a Docker container, for a simple toy C++ API that
uses both `std::vector` and `Eigen::Matrix<>`. The code is defined inline.

In [ ]:
from pygccxml import declarations
from pygccxml import utils
from pygccxml import parser

In [ ]:
# Find out the c++ parser. This should resolve to the castxml
# version installed in Docker.
generator_path, generator_name = utils.find_xml_generator()

# Configure the xml generator
config = parser.xml_generator_configuration_t(
    xml_generator_path=generator_path,
    xml_generator=generator_name,
    include_paths=["/usr/include/eigen3"],
    # TODO(eric.cousineau): Why is `compiler_path` necessary?
    compiler_path=generator_path,
)

In [ ]:
code = r"""
#include <vector>

#include <Eigen/Dense>

namespace ns {

template <typename T, typename U = int>
class ExampleClass {
public:
    std::vector<T> make_std_vector() const;
    Eigen::Matrix<U, 3, 3> make_matrix3();
};

// Analyze concrete instantiations of the given class.
extern template class ExampleClass<int>;
extern template class ExampleClass<float, float>;

}  // namespace ns
"""

(global_ns,) = parser.parse_string(code, config)

In [ ]:
ns = global_ns.namespace("ns")
declarations.print_declarations([ns])

In [ ]:
# Retrieve an instantiation and show template parameters.
cls, = ns.classes('ExampleClass<float, float>')
declarations.templates.split(cls.name)